# Matrix Factorization

In a recommendation system, there is a group of users and a set of items. Given that each users have rated some items in the system, we would like to predict how the users would rate the items that they have not yet rated, such that we can make recommendations to the users.

Matrix factorization is one of the mainly used algorithm in recommendation systems. It can be used to discover latent features underlying the interactions between two different kinds of entities. Assume we assign a $k$ dimensional vector $u_i$ to user $i$ and $k$ dimensional vector $v_j$ to item $j$, then user $i$ rates movie $j$ by $\langle u_i, v_j\rangle$.

We can learn all $u_i$ and $v_j$ directly, which is essentially performing SVD on the user-item matrix. We can also try to learn the latent features using multi-layer neural networks. 

In this tutorial, we will work though the steps to implement these ideas in MXNet.

## Prepare Data

We use the [MovieLens](http://grouplens.org/datasets/movielens/) data here, but it can apply to other datasets as well. Each row of this dataset contains a tuple of user id, movie id, rating, and time stamp, we will only use the first three items. We first define the a batch which contains n tuples. It also provides name and shape information to MXNet about the data and label. 

In [9]:
class Batch(object):
    def __init__(self, data_names, data, label_names, label):
        self.data = data
        self.label = label
        self.data_names = data_names
        self.label_names = label_names
        
    @property
    def provide_data(self):
        return [(n, x.shape) for n, x in zip(self.data_names, self.data)]
    
    @property
    def provide_label(self):
        return [(n, x.shape) for n, x in zip(self.label_names, self.label)]


Then we define a data iterator, which returns a batch of tuples each time. 

In [10]:
import mxnet as mx
import random

class Batch(object):
    def __init__(self, data_names, data, label_names, label):
        self.data = data
        self.label = label
        self.data_names = data_names
        self.label_names = label_names

    @property
    def provide_data(self):
        return [(n, x.shape) for n, x in zip(self.data_names, self.data)]

    @property
    def provide_label(self):
        return [(n, x.shape) for n, x in zip(self.label_names, self.label)]

class DataIter(mx.io.DataIter):
    def __init__(self, fname, batch_size):
        super(DataIter, self).__init__()
        self.batch_size = batch_size
        self.data = []
        for line in file(fname):
            tks = line.strip().split('\t')
            if len(tks) != 4:
                continue
            self.data.append((int(tks[0]), int(tks[1]), float(tks[2])))
        self.provide_data = [('user', (batch_size, )), ('item', (batch_size, ))]
        self.provide_label = [('score', (self.batch_size, ))]

    def __iter__(self):
        for k in range(len(self.data) / self.batch_size):
            users = []
            items = []
            scores = []
            for i in range(self.batch_size):
                j = k * self.batch_size + i
                user, item, score = self.data[j]
                users.append(user)
                items.append(item)
                scores.append(score)

            data_all = [mx.nd.array(users), mx.nd.array(items)]
            label_all = [mx.nd.array(scores)]
            data_names = ['user', 'item']
            label_names = ['score']

            data_batch = Batch(data_names, data_all, label_names, label_all)
            yield data_batch

    def reset(self):
        random.shuffle(self.data)

Now we download the data and provide a function to obtain the data iterator:

In [11]:
import os
if not os.path.exists("ml-100k.zip"):
    os.system("wget http://files.grouplens.org/datasets/movielens/ml-100k.zip") 
    os.system("unzip ml-100k.zip")
def get_data(batch_size):
    return (DataIter('./ml-100k/u1.base', batch_size), DataIter('./ml-100k/u1.test', batch_size))

Finally we calculate the numbers of users and items for later use.

In [12]:
def max_id(fname):
    mu = 0
    mi = 0
    for line in file(fname):
        tks = line.strip().split('\t')
        if len(tks) != 4:
            continue
        mu = max(mu, int(tks[0]))
        mi = max(mi, int(tks[1]))
    return mu + 1, mi + 1
max_user, max_item = max_id('./ml-100k/u.data')
(max_user, max_item)

(944, 1683)

## Optimization

We first implement the RMSE (root-mean-square error) measurement, which is commonly used by matrix factorization. 

In [13]:
import math
def RMSE(label, pred):
    ret = 0.0
    n = 0.0
    pred = pred.flatten()
    for i in range(len(label)):
        ret += (label[i] - pred[i]) * (label[i] - pred[i])
        n += 1.0
    return math.sqrt(ret / n)

Then we define a general training module, which is borrowed from the image classification application. 

In [14]:
def train(network, batch_size, num_epoch, learning_rate):
    model = mx.model.FeedForward(
        ctx = [mx.gpu(0)],  # can be change to [mx.gpu(0), mx.gpu(1)] if there are 2 gpus
        symbol = network,
        num_epoch = num_epoch,
        learning_rate = learning_rate,
                             #lr_scheduler = mx.lr_scheduler.FactorScheduler(
        #step = max(int(80000/batch_size*2), 1),
         #   factor = 0.95),
        wd = 0.0001,
        #initializer = mx.init.Xavier(factor_type="in", magnitude=2.34),
        momentum = 0.9)

    batch_size = 64
    train, test = get_data(batch_size)

    import logging
    head = '%(asctime)-15s %(message)s'
    logging.basicConfig(level=logging.DEBUG)

    model.fit(X = train, 
              eval_data = test,
              eval_metric = RMSE,
              batch_end_callback=mx.callback.Speedometer(batch_size, 10000/batch_size),)

## Networks

Now we try various networks. We first learn the latent vectors directly.

In [15]:
def plain_net(k):
    # input
    user = mx.symbol.Variable('user')
    item = mx.symbol.Variable('item')
    score = mx.symbol.Variable('score')
    # user feature lookup
    user = mx.symbol.Embedding(data = user, input_dim = max_user, output_dim = k) 
    # item feature lookup
    item = mx.symbol.Embedding(data = item, input_dim = max_item, output_dim = k)
    # predict by the inner product, which is elementwise product and then sum
    pred = user * item
    pred = mx.symbol.sum_axis(data = pred, axis = 1)
    pred = mx.symbol.Flatten(data = pred)
    # loss layer
    pred = mx.symbol.LinearRegressionOutput(data = pred, label = score)
    return pred

train(plain_net(64), batch_size=64, num_epoch=10, learning_rate=.05)

INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [156]	Speed: 56152.02 samples/sec	Train-RMSE=3.706615
INFO:root:Epoch[0] Batch [312]	Speed: 56093.35 samples/sec	Train-RMSE=3.697545
INFO:root:Epoch[0] Batch [468]	Speed: 55999.96 samples/sec	Train-RMSE=3.708319
INFO:root:Epoch[0] Batch [624]	Speed: 56241.24 samples/sec	Train-RMSE=3.688322
INFO:root:Epoch[0] Batch [780]	Speed: 55798.34 samples/sec	Train-RMSE=3.704836
INFO:root:Epoch[0] Batch [936]	Speed: 55368.81 samples/sec	Train-RMSE=3.683681
INFO:root:Epoch[0] Batch [1092]	Speed: 55267.68 samples/sec	Train-RMSE=3.705261
INFO:root:Epoch[0] Batch [1248]	Speed: 54383.48 samples/sec	Train-RMSE=3.694059
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=1.477
INFO:root:Epoch[0] Validation-RMSE=3.713718
INFO:root:Epoch[1] Batch [156]	Speed: 55056.23 samples/sec	Train-RMSE=3.697979
INFO:root:Epoch[1] Batch [312]	Speed: 54194.29 samples/sec	Train-RMSE=3.668891
INFO:root:Epoch[1] Batch [468]	Speed: 54458.23 

Next we try to use 2 layers neural network to learn the latent variables, which stack a fully connected layer above the embedding layers: 

In [16]:
def get_one_layer_mlp(hidden, k):
    # input
    user = mx.symbol.Variable('user')
    item = mx.symbol.Variable('item')
    score = mx.symbol.Variable('score')
    # user latent features
    user = mx.symbol.Embedding(data = user, input_dim = max_user, output_dim = k)
    user = mx.symbol.FullyConnected(data = user, num_hidden = hidden)
    # item latent features
    item = mx.symbol.Embedding(data = item, input_dim = max_item, output_dim = k)
    item = mx.symbol.FullyConnected(data = item, num_hidden = hidden)
    # predict by the inner product
    pred = user * item
    pred = mx.symbol.sum_axis(data = pred, axis = 1)
    pred = mx.symbol.Flatten(data = pred)
    # loss layer
    pred = mx.symbol.LinearRegressionOutput(data = pred, label = score)
    return pred

train(get_one_layer_mlp(64, 64), batch_size=64, num_epoch=10, learning_rate=.02)

INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [156]	Speed: 34221.56 samples/sec	Train-RMSE=1.942634
INFO:root:Epoch[0] Batch [312]	Speed: 30912.80 samples/sec	Train-RMSE=1.119005
INFO:root:Epoch[0] Batch [468]	Speed: 35044.73 samples/sec	Train-RMSE=1.098399
INFO:root:Epoch[0] Batch [624]	Speed: 35098.19 samples/sec	Train-RMSE=1.054476
INFO:root:Epoch[0] Batch [780]	Speed: 35128.96 samples/sec	Train-RMSE=1.011925
INFO:root:Epoch[0] Batch [936]	Speed: 35080.08 samples/sec	Train-RMSE=0.983108
INFO:root:Epoch[0] Batch [1092]	Speed: 34945.15 samples/sec	Train-RMSE=0.977220
INFO:root:Epoch[0] Batch [1248]	Speed: 33966.69 samples/sec	Train-RMSE=0.965951
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=2.376
INFO:root:Epoch[0] Validation-RMSE=0.978114
INFO:root:Epoch[1] Batch [156]	Speed: 30683.17 samples/sec	Train-RMSE=0.949212
INFO:root:Epoch[1] Batch [312]	Speed: 30868.07 samples/sec	Train-RMSE=0.951608
INFO:root:Epoch[1] Batch [468]	Speed: 31483.75 

Adding dropout layers to relief the over-fitting. 

In [17]:
def get_one_layer_dropout_mlp(hidden, k):
    # input
    user = mx.symbol.Variable('user')
    item = mx.symbol.Variable('item')
    score = mx.symbol.Variable('score')
    # user latent features
    user = mx.symbol.Embedding(data = user, input_dim = max_user, output_dim = k)
    user = mx.symbol.FullyConnected(data = user, num_hidden = hidden)
    user = mx.symbol.Dropout(data=user, p=0.5)
    # item latent features
    item = mx.symbol.Embedding(data = item, input_dim = max_item, output_dim = k)
    item = mx.symbol.FullyConnected(data = item, num_hidden = hidden)
    item = mx.symbol.Dropout(data=item, p=0.5)    
    # predict by the inner product
    pred = user * item
    pred = mx.symbol.sum_axis(data = pred, axis = 1)
    pred = mx.symbol.Flatten(data = pred)
    # loss layer
    pred = mx.symbol.LinearRegressionOutput(data = pred, label = score)
    return pred
train(get_one_layer_mlp(256, 512), batch_size=64, num_epoch=10, learning_rate=.02)

INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [156]	Speed: 29301.10 samples/sec	Train-RMSE=1.790682
INFO:root:Epoch[0] Batch [312]	Speed: 29281.08 samples/sec	Train-RMSE=1.113848
INFO:root:Epoch[0] Batch [468]	Speed: 29358.50 samples/sec	Train-RMSE=1.060094
INFO:root:Epoch[0] Batch [624]	Speed: 29291.55 samples/sec	Train-RMSE=1.005032
INFO:root:Epoch[0] Batch [780]	Speed: 29233.50 samples/sec	Train-RMSE=0.975966
INFO:root:Epoch[0] Batch [936]	Speed: 29310.04 samples/sec	Train-RMSE=0.982252
INFO:root:Epoch[0] Batch [1092]	Speed: 29440.38 samples/sec	Train-RMSE=0.972627
INFO:root:Epoch[0] Batch [1248]	Speed: 29424.34 samples/sec	Train-RMSE=0.972099
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=2.768
INFO:root:Epoch[0] Validation-RMSE=0.983640
INFO:root:Epoch[1] Batch [156]	Speed: 22813.32 samples/sec	Train-RMSE=0.944569
INFO:root:Epoch[1] Batch [312]	Speed: 22577.30 samples/sec	Train-RMSE=0.950992
INFO:root:Epoch[1] Batch [468]	Speed: 22614.69 

## Acknowledgement

This tutorial is based on examples from [xlvector/github](https://github.com/xlvector/).